## Setup

In [2]:
import numpy as np
import pandas as pd
from scipy.constants import c
from scipy.optimize import minimize
import csv
import sys
import os
import threading, time, ctypes
from pywinauto import Desktop
import matplotlib.pyplot as plt
import textwrap

# ── Custom optimizer stack (Normalizer / Bridge / Stepper / Gradient / Preconditioner / Memory)
from Normalize import boundary                       # x∈[L,U]^d ↔ y∈[0,1]^d 매핑
from Bridge import OnePointOptimizer, BridgeConfig   # 단일-루프 최적화 오케스트레이터
from Stepper import StepperConfig, AdamState         # AdamW×Muon + TR/백트래킹 정책
from preconditioner import PreconditionerConfig      # 좌표 스케일/회전 안정화
from Gradient import Forward1P, Central2P            # 유한차분 1P/2P 그라디언트 소스
from Memory import MemoryMetric, MemoryEvent         # 수용/진동 신호 기록
import math, time


ModuleNotFoundError: No module named 'Normalize'

In [ ]:
lumapi_path = r"C:\Program Files\Lumerical\v251\api\python"
if lumapi_path not in sys.path:
    sys.path.append(lumapi_path)

import lumapi

C:\Program Files\Lumerical\v251\api\python\lumapi.py:895: SyntaxWarning: invalid escape sequence '\s'
  message = re.sub('^(Error:)\s(prompt line)\s[0-9]+:', '', str(rvals[2])).strip()


In [ ]:
mm = 1e-3
um = 1e-6
nm = 1e-9

In [ ]:
_thread_started = False

def remove_popup():
    global _thread_started
    if _thread_started: return
    _thread_started = True

    def _worker():
        d = Desktop(backend='uia')
        while True:
            for w in d.windows():
                if w.window_text().strip() == "Select frequency":
                    ctypes.windll.user32.AllowSetForegroundWindow(-1)
                    try:
                        w.minimize(); time.sleep(0.1); w.restore(); time.sleep(0.1)
                    except: pass
                    for b in w.descendants(control_type="Button"):
                        if b.window_text() == "All":
                            try: b.invoke()
                            except: b.click_input()
                            break
                        elif b.window_text() == "OK":
                            try: b.invoke()
                            except: b.click_input()
                            break

    threading.Thread(target=_worker, daemon=True).start()

In [ ]:
def get_mon(fdtd, mon:str):
    # raw_datasets
    raw_datasets = fdtd.getresult(mon)   # e.g. ["Ex", "\n", "Ey", "\n", "Ez", ...]

    # integrating string
    combined = "".join(raw_datasets)      # "Ex\nEy\nEz..."

    # organize label
    datasets = [ds for ds in combined.split("\n") if ds]  

    # data extract
    results = {}
    for ds in datasets:
        try:
            remove_popup()
            results[ds] = fdtd.getresult(mon, ds)
        except Exception:
            pass

    # organize dataframe
    df_dict = {}
    length = None
    for key, arr in results.items():
        arr1d = np.array(arr).ravel()
        if arr1d.ndim == 1:
            if length is None:
                length = arr1d.shape[0]
            if arr1d.shape[0] == length:
                df_dict[key] = arr1d

    df = pd.DataFrame(df_dict)
    return df

## Plot code

In [ ]:
def plot_map(df, key=None, comp=None, norm=True, sum_sq=False):
    """
    df     : data frame
    key    : keyword of dataframe label
    comp   : component of data ; Polarization, Direction
    norm   : Normalization data
    sum_sq : Sum sequence
    """

    if key not in df.columns:
        TypeError(f"ERROR: '{key}' No column"); return
    entry = df[key].iloc[0]
    x = np.array(entry.get('x', []))
    y = np.array(entry.get('y', []))
    data3d = np.squeeze(np.array(entry[key]))  # (Nx,Ny,3)
    if data3d.ndim!=3 or data3d.shape[2]!=3:
        TypeError("Data shape error:", data3d.shape); return

    if sum_sq:
        Z = np.sum(np.abs(data3d)**2, axis=2)
        title, cbar = f"{key} sum_sq", "sum_sq"
    elif norm:
        Z = np.linalg.norm(data3d, axis=2)
        title, cbar = f"{key} norm", "norm"
    else:
        c = comp or 0
        Z = data3d[:,:,c]
        title, cbar = f"{key}[{c}]", f"comp{c}"

    if np.iscomplexobj(Z):
        Z = np.abs(Z)

    X, Y = np.meshgrid(x, y, indexing='ij')
    plt.figure(figsize=(6,5))
    pcm = plt.pcolormesh(X, Y, Z, shading='auto', cmap='magma')
    plt.colorbar(pcm, label=cbar)
    plt.xlabel('x (m)'); plt.ylabel('y (m)'); plt.title(title)
    plt.tight_layout(); plt.show()

In [ ]:
def plot_farfield(ff, lam_idx=0, cmap='magma'):
    """
    Far-field 
    ff : data of farfield
    lam_idx : index of wavelength
    cmap : name of colormap
    """
    # 방향 코사인
    ux = np.array(ff['ux'])
    uy = np.array(ff['uy'])
    # intensity 및 편광
    E2_all = np.array(ff['E2'])    # shape (Nux, Nuy, 1, Nλ)
    Ep_all = np.array(ff['Ep'])
    Es_all = np.array(ff['Es'])
    
    # 불필요 차원 제거 및 파장 인덱스 선택
    E2 = np.squeeze(E2_all[:, :, 0, lam_idx])
    Ep = np.squeeze(Ep_all[:, :, 0, lam_idx])
    Es = np.squeeze(Es_all[:, :, 0, lam_idx])
    
    # 2D 맵 계산
    E2_map = np.real(E2)
    Ep_map = np.abs(Ep)
    Es_map = np.abs(Es)
    
    # 그리드 생성
    X, Y = np.meshgrid(ux, uy, indexing='ij')
    
    # 서브플롯
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    titles = ['Total Intensity (E2)', 'P-Polarized |Ep|', 'S-Polarized |Es|']
    maps   = [E2_map, Ep_map, Es_map]
    labels = ['E2', '|Ep|', '|Es|']
    
    for ax, Z, title, lbl in zip(axes, maps, titles, labels):
        pcm = ax.pcolormesh(X, Y, Z, shading='auto', cmap=cmap)
        ax.set_title(title)
        ax.set_xlabel('ux')
        ax.set_ylabel('uy')
        fig.colorbar(pcm, ax=ax, label=lbl)
    
    plt.tight_layout()
    plt.show()

## Operation

In [ ]:
def analyze_farfield(mon: dict, src: dict, filter_type: str='gauss', NA: float=0.12, threshold: float=0.95) -> dict:
    import numpy as np

    # ---------- helpers ----------
    def _radial_filter(rho, kind, scale):
        if kind == 'flat':    return (rho <= scale).astype(float)
        if kind == 'gauss':   return np.exp(-(rho*rho)/(2.0*scale*scale))
        if kind == 'sech':    return 1.0/np.cosh(rho/scale)
        if kind == 'lorentz': return 1.0/(1.0 + (rho/scale)**2)
        raise ValueError(f"Unknown filter_type: {kind}")

    def _scale_for_cover(rho, NA, cover, kind):
        rho   = np.asarray(rho)
        NA    = float(np.clip(NA, 0.0, 1.0))
        cover = float(np.clip(cover, 0.0, 1.0))
        mNA   = (rho <= NA)
        mall  = (rho <= 1.0)
        def frac(s):
            F = _radial_filter(rho, kind, s)
            num = float((F * mNA ).sum())
            den = float((F * mall).sum())
            return 0.0 if den <= 0.0 else (num/den)
        lo, hi = (0.0, 1.0) if kind=='flat' else (1e-6, max(5.0*max(1e-3, NA), 1.0))
        for _ in range(64):
            mid = 0.5*(lo+hi)
            if frac(mid) < cover: hi = mid
            else:                  lo = mid
        return 0.5*(lo+hi)

    def _get_complex(ffdict, base):
        r, i = base+'_r', base+'_i'
        if (r in ffdict) and (i in ffdict):
            return np.asarray(ffdict[r]) + 1j*np.asarray(ffdict[i])
        elif base in ffdict:
            return np.asarray(ffdict[base])
        return None

    def _slice_best_2d(A, best_idx):
        A = np.asarray(A)
        sel = A[..., best_idx]
        # (최소한의 축 합: 동일 결과, 불필요한 누적 제거)
        while sel.ndim > 2:
            sel = sel.sum(axis=0)
        return sel  # (Nx, Ny)

    def _ascend(x, y=None):
        x = np.asarray(x).ravel()
        if x.size >= 2 and x[0] > x[-1]:
            x = x[::-1]
            if y is None:
                return x, None
            if isinstance(y, (list, tuple)):
                return x, [np.asarray(v)[::-1] for v in y]
            return x, np.asarray(y)[::-1]
        return x, y

    # NEW: 안전 1D 보간기 -----------------------------------------------
    def _interp1d_safe(x, xp, fp):
        """np.interp 래퍼: 1D float 강제, NaN 제거, 정렬/중복 처리."""
        x  = np.asarray(x,  dtype=float).ravel()
        xp = np.asarray(xp, dtype=float).ravel()
        fp = np.asarray(fp, dtype=float).ravel()

        m = np.isfinite(xp) & np.isfinite(fp)
        xp, fp = xp[m], fp[m]
        if xp.size == 0:
            return np.full_like(x, np.nan)
        if xp.size == 1:
            return np.full_like(x, fp[0])

        # 오름차순 정렬
        idx = np.argsort(xp, kind='mergesort')
        xp, fp = xp[idx], fp[idx]

        # 중복 xp 평균병합
        if xp.size > 1 and np.any(np.diff(xp) == 0):
            xu, inv = np.unique(xp, return_inverse=True)
            fp_sum = np.zeros_like(xu)
            cnt    = np.zeros_like(xu)
            np.add.at(fp_sum, inv, fp)
            np.add.at(cnt,    inv, 1)
            xp, fp = xu, fp_sum / np.maximum(cnt, 1)

        return np.interp(x, xp, fp)
    # -------------------------------------------------------------------

    # --- 0) farfield grid & rho ---
    ff  = mon['farfield'][0]
    ux  = np.asarray(ff['ux']).astype(float).ravel()
    uy  = np.asarray(ff['uy']).astype(float).ravel()
    Ux, Uy = np.meshgrid(ux, uy, indexing='ij')
    rho    = np.hypot(Ux, Uy)

    # --- 1) intensity cube ---
    E2    = np.asarray(ff['E2'], dtype=float)  # (Nx,Ny,Nz,Nf)
    I_sum = E2.sum(axis=2)                     # (Nx,Ny,Nf)
    Nx, Ny, Nf = I_sum.shape
    V = np.ascontiguousarray(I_sum.reshape(Nx*Ny, Nf))   # (M,Nf)
    sums_total = V.sum(axis=0).astype(float)
    sums_total[sums_total == 0] = np.nan

    # --- 2) filters ---
    cover = threshold
    def _build_filter(rho, kind, NA, cover):
        if kind == 'flat':
            Fk = (rho <= NA).astype(float); scale = NA
        else:
            scale = _scale_for_cover(rho, NA, cover, kind)
            Fk = _radial_filter(rho, kind, scale).astype(float)
        return Fk.ravel(), float(scale)

    F_g, _ = _build_filter(rho, 'gauss',   NA, cover)
    F_l, _ = _build_filter(rho, 'lorentz', NA, cover)
    F_s, _ = _build_filter(rho, 'sech',    NA, cover)
    F_f, _ = _build_filter(rho, 'flat',    NA, cover)

    # --- 3) CE spectra (단일 matmul) ---
    F_mat = np.ascontiguousarray(np.stack([F_g, F_l, F_s, F_f], axis=1))  # (M,4)
    CE_all = (V.T @ F_mat) / sums_total[:, None]                           # (Nf,4)
    CE_gauss_spec, CE_lorentz_spec, CE_sech_spec, CE_flat_spec = (
        CE_all[:,0], CE_all[:,1], CE_all[:,2], CE_all[:,3]
    )

    # --- 4) wavelength alignment ---
    lam_P, PUR = _ascend(src['purcell'][0]['lambda'], src['purcell'][0]['purcell'])
    lam_P = np.asarray(lam_P, dtype=float).ravel()
    PUR   = np.asarray(PUR,   dtype=float).ravel()

    lam_FF, CE_stack = _ascend(ff['lambda'], [CE_gauss_spec, CE_lorentz_spec, CE_sech_spec, CE_flat_spec])
    lam_FF = np.asarray(lam_FF, dtype=float).ravel()
    CE_gauss_spec, CE_lorentz_spec, CE_sech_spec, CE_flat_spec = [np.asarray(a, dtype=float).ravel() for a in CE_stack]

    # 안전 보간기로 교체 (여기가 에러 지점)  -----------------------------
    CE_g    = _interp1d_safe(lam_P, lam_FF, CE_gauss_spec)
    CE_l    = _interp1d_safe(lam_P, lam_FF, CE_lorentz_spec)
    CE_s    = _interp1d_safe(lam_P, lam_FF, CE_sech_spec)
    CE_flat = _interp1d_safe(lam_P, lam_FF, CE_flat_spec)
    # -------------------------------------------------------------------

    # --- 5) Transmission (raw) ---
    lam_T, T_raw = _ascend(mon['T'][0]['lambda'], mon['T'][0]['T'])
    lam_T = np.asarray(lam_T, dtype=float).ravel()
    T_raw = np.asarray(T_raw, dtype=float).ravel()
    T_on_P = _interp1d_safe(lam_P, lam_T, T_raw)
    T_norm_sp = np.maximum(T_on_P, 0.0)

    # --- 6) best wavelength ---
    best_idx = int(np.nanargmax(PUR))
    best_lam = float(lam_P[best_idx])
    ff_idx   = int(np.nanargmin(np.abs(lam_FF - best_lam)))

    # --- 7) Stokes ---
    Ep_all = _get_complex(ff, 'Ep')
    Es_all = _get_complex(ff, 'Es')
    S_0 = S_1 = S_2 = S_3 = np.nan
    DoLP = DoCP = np.nan
    if (Ep_all is not None) and (Es_all is not None):
        Epb = _slice_best_2d(Ep_all, ff_idx)
        Esb = _slice_best_2d(Es_all, ff_idx)
        Esb_conj = np.conj(Esb)
        abs_Epb2 = np.abs(Epb)**2
        abs_Esb2 = np.abs(Esb)**2
        cross    = Epb * Esb_conj
        mNA = (rho <= NA)
        S_0 = float(((abs_Epb2 + abs_Esb2) * mNA).sum())
        S_1 = float(((abs_Epb2 - abs_Esb2) * mNA).sum())
        S_2 = float(((2.0*np.real(cross)) * mNA).sum())
        S_3 = float(((-2.0*np.imag(cross)) * mNA).sum())
        eps = 1e-300
        inv_S0 = 1.0 / (S_0 + eps)
        DoLP = np.sqrt(S_1**2 + S_2**2) * inv_S0
        DoCP = S_3 * inv_S0

    # --- 8) return ---
    return {
        'best_wl_idx': best_idx,
        'lambda':      best_lam,
        'CE_g':        float(CE_g[best_idx]),
        'CE_l':        float(CE_l[best_idx]),
        'CE_s':        float(CE_s[best_idx]),
        'CE_flat':     float(CE_flat[best_idx]),
        'T_norm':      float(T_norm_sp[best_idx]),
        'purcell':     float(PUR[best_idx]),
        'S_0':         S_0,
        'S_1':         S_1,
        'S_2':         S_2,
        'S_3':         S_3,
        'DoLP':        DoLP,
        'DoCP':        DoCP,
    }


In [ ]:
# -*- coding: utf-8 -*-
# Self-contained helpers (NumPy + lumapi; assumes your get_mon/analyze_farfield exist)
import numpy as np
import lumapi

um = 1e-6  # micrometer in meters


def _S_mode(t: float, mode: str = "arch", gamma: float = 1.0) -> float:
    """
    Monotone shaping map S:[0,1]→[0,1] used for radial spiral profiles.
    Inputs:
      t     : normalized radius in [0,1]
      mode  : {'none','arch','log','clothoid'} profile family
      gamma : exponent shaping (>0), applied as S^gamma
    Output:
      float in [0,1]
    """
    t = max(0.0, min(1.0, float(t)))
    if mode == "none":
        S0 = 1.0
    elif mode == "arch":
        S0 = t
    elif mode == "log":
        S0 = np.log(1.0 + (np.e - 1.0) * t)
    elif mode == "clothoid":
        a, p = 1.0, 1.0
        q = p + 1.0
        S0 = t if abs(q) < 1e-12 else ((1.0 + a * t) ** q - 1.0) / ((1.0 + a) ** q - 1.0)
    else:
        S0 = t
    g = max(float(gamma), 1e-9)
    return float(np.clip(S0, 0.0, 1.0)) ** g


def make_spiral_offsets(
    ring_num: int,
    *,
    base_even: float = 0.0, base_odd: float = 0.0,   # constant (× step)
    amp_even: float = 0.0,  amp_odd: float = 0.0,    # amplitude (× step)
    mode: str = "arch", gamma: float = 1.0
) -> tuple[list[float], list[float]]:
    """
    Create per-ring angular offsets (× step) for even/odd rings following a simple spiral profile.
    Returns:
      (e_eta_seq, o_eta_seq): lists of length=ring_num. Use with `geom(..., e_eta_seq=..., o_eta_seq=...)`.
    """
    if ring_num <= 0:
        return [], []
    e_seq, o_seq = [], []
    for i in range(ring_num):
        t = 0.0 if ring_num == 1 else i / (ring_num - 1)  # normalized radius
        S = _S_mode(t, mode=mode, gamma=gamma)
        e_seq.append(base_even + amp_even * S)
        o_seq.append(base_odd  + amp_odd  * S)
    return e_seq, o_seq


def geom(
    a_phi_m: float, hole_radius_m: float, N0: int, delta_N: int, ring_num: int,
    wl_i_m: float, wl_f_m: float,
    *,
    # per-ring offsets (×step) from make_spiral_offsets()
    e_eta_seq: list[float] | None = None,
    o_eta_seq: list[float] | None = None,
    # parity & radius compensation
    parity_mat: bool = True,       # True: odd rings remove exactly one hole (2n→2n-1)
    rho: float = 0.5,              # r' = r * (N_after/N_before)**rho
    # IO
    NA: float = 0.12,
    f_name: str = "temp",
    slab_material: str = "InP - Palik",
    dpml: float = 0.5 * um,
    slab_thickness: float = 0.3 * um,
):
    """
    Build and run a hole-type circular Bragg/PhC ring with per-ring spiral offsets (Lumerical FDTD).
    Inputs:
      - geometry and wavelength band in meters; ring parity and radius compensation controls.
    Output:
      - Whatever your `analyze_farfield(ff, src, NA=...)` returns (dict of metrics expected).
    Notes:
      - Requires helper functions `get_mon(session,name)` and `analyze_farfield(ff, src, NA=...)`.
      - Updated to current APIs: uses adddftmonitor (replacing deprecated addprofile), sets background index on FDTD.
    """
    # ---------- derived geometry ----------
    R0      = (N0 * a_phi_m) / (2 * np.pi)
    delta_r = (delta_N * a_phi_m) / (2 * np.pi)
    r_max   = R0 + delta_r * (ring_num - 1)
    span_xy = (r_max + dpml) * 2 + dpml

    # ---------- sanitize offset sequences ----------
    e_eta_seq = (e_eta_seq or [0.0] * ring_num)[:ring_num]
    o_eta_seq = (o_eta_seq or [0.0] * ring_num)[:ring_num]

    # ---------- Lumerical session ----------
    # Use supported constructor + serverArgs (offscreen solve).  [Docs: class FDTD + serverArgs]
    fdtd = lumapi.FDTD(hide=False)

    try:
        # Region
        fdtd.newproject()
        fdtd.addfdtd()                                                # add FDTD region (3D default)  [doc]
        fdtd.setnamed("FDTD", "x span", span_xy)
        fdtd.setnamed("FDTD", "y span", span_xy)
        fdtd.setnamed("FDTD", "z span", 4e-6)
        fdtd.setnamed("FDTD", "index", 1.0)                # not "index" on solver  [doc]

        # Slab
        fdtd.addcircle(); fdtd.set("name", "slab")                    # circle primitive (extruded by z span)  [doc]
        fdtd.set("x", 0); fdtd.set("y", 0)
        fdtd.set("radius", r_max + dpml)
        fdtd.set("z span", slab_thickness)
        fdtd.set("material", slab_material)

        # Holes as a scripted group (fast object creation)
        script = ['addgroup; set("name","holes");']
        for i in range(ring_num):
            ring_idx = i + 1
            Rm = R0 + i * delta_r
            Nb = max(int(N0 + i * delta_N), 1)               # baseline count (even: 2n)
            is_odd_ring = (ring_idx % 2 == 1)
            Na = (Nb - 1) if (parity_mat and is_odd_ring and Nb > 1) else Nb

            step    = 2 * np.pi / Na
            pitch_i = (2 * np.pi * Rm) / Na
            theta0  = (o_eta_seq[i] if is_odd_ring else e_eta_seq[i]) * step

            # radius compensation and simple collision guard
            r_use = hole_radius_m * (Na / Nb) ** float(rho) if Na != Nb else hole_radius_m
            if 2 * r_use > 0.9 * pitch_i:
                r_use = 0.45 * pitch_i

            for j in range(Na):
                th = theta0 + j * step
                x = Rm * np.cos(th); y = Rm * np.sin(th)
                script.append(
                    "addcircle;"
                    f'set("x",{x}); set("y",{y}); set("z span",{slab_thickness}); '
                    f'set("radius",{r_use}); '
                    'set("material","<Object defined dielectric>"); set("index",1.0); '
                    'addtogroup("holes");'
                )
        fdtd.eval("\n".join(script))

        # Source (dipole)
        fdtd.adddipole(); fdtd.set("name", "src")                     # dipole source  [doc]
        fdtd.set("theta", 90); fdtd.set("phi", 90)
        fdtd.set("x", 0); fdtd.set("y", 0); fdtd.set("z", 0)
        fdtd.set("wavelength start", wl_i_m)
        fdtd.set("wavelength stop",  wl_f_m)
        fdtd.set("amplitude", 1)

        # DFT field monitor (replaces deprecated addprofile)
        fdtd.adddftmonitor(); fdtd.set("name", "far")                 # DFT monitor  [doc]
        fdtd.set("monitor type", 7)                                   # 2D z-normal  [doc]
        fdtd.set("override global monitor settings", True)
        fdtd.set("x", 0); fdtd.set("y", 0); fdtd.set("z", 1e-6)
        fdtd.set("x span", span_xy); fdtd.set("y span", span_xy)
        fdtd.set("frequency points", 100)

        # BC + far-field settings
        fdtd.setnamed("FDTD", "z min bc", "Symmetric")
        fdtd.eval('farfieldsettings("override near field mesh",1);')   # global FF settings  [doc]
        fdtd.eval('farfieldsettings("near field samples per wavelength",100);')

        # Run
        fdtd.save(f_name)
        fdtd.run()

        # Extract
        ff  = get_mon(fdtd, "far")
        src = get_mon(fdtd, "src")
    finally:
        try:
            fdtd.close()
        except Exception:
            pass

    # Post
    res = analyze_farfield(ff, src, NA=NA)
    return res


res, src = geom(
    a_phi=0.136*um, hole_radius=0.0466*um,
    delta_N=8, N0=16, ring_num=12,
    wavelength_i=1.28e-6, wavelength_f=1.32e-6,
    eta=0.5, rev_tar="even"
)

In [ ]:
"""
Photonic-structure optimizer (CPU-only, single-process)
- External interface in nm
- Physics called in meters (nm→m inside objective)
- Custom optimizer stack: Normalizer + Bridge + Stepper + Gradient + (optional) Preconditioner
- Always logs 4 CSV files (params, analysis, FD-gradient samples, zoom)
- Prints one human-readable line per evaluation (flush=True) when disp=True
"""

# ───────────────────────────────────────────────────────────────
# Imports
# ───────────────────────────────────────────────────────────────
import os, csv, time, hashlib, traceback
import numpy as np
import pandas as pd
from threading import RLock

# Custom optimizer stack (must exist in working directory)
from Normalize import boundary                               # x∈[L,U]^d ↔ y∈[0,1]^d
from Bridge import OnePointOptimizer, BridgeConfig           # loop orchestrator
from Stepper import StepperConfig, AdamState                 # AdamW×Muon + trust-region policy
from preconditioner import PreconditionerConfig              # SPD scaling/rotation (optional)
from Gradient import Forward1P, Central2P                    # 1P bootstrap → 2P central FD

# ───────────────────────────────────────────────────────────────
# Thread-safe (single process) cache & file I/O
# ───────────────────────────────────────────────────────────────
_OBJ_CACHE   = {}          # key -> (obj, res)
_CACHE_ROUND = 14
_CACHE_MAX   = 4096
_CACHE_LOCK  = RLock()
_FILE_LOCK   = RLock()

def clear_cache():
    """Clear all objective cache entries (global)."""
    with _CACHE_LOCK:
        _OBJ_CACHE.clear()

def _digest_seq(seq) -> str:
    """Stable short id for lists/tuples/None used in cache & logs."""
    if seq is None:
        return "none"
    arr = np.asarray(seq, float).ravel()
    return hashlib.sha1(arr.tobytes()).hexdigest()[:10]

def _obj_key(x, N0, delta_N, ring_num, wl_i, wl_f, f_name,
             *, e_eta_seq=None, o_eta_seq=None, parity_mat=True, rho=0.5, NA=0.12):
    a_phi, hole_radius = map(float, x[:2])
    return (
        round(a_phi, _CACHE_ROUND), round(hole_radius, _CACHE_ROUND),
        int(N0), int(delta_N), int(ring_num),
        round(float(wl_i), _CACHE_ROUND), round(float(wl_f), _CACHE_ROUND),
        str(f_name),
        _digest_seq(e_eta_seq), _digest_seq(o_eta_seq),
        bool(parity_mat), round(float(rho), _CACHE_ROUND),
        round(float(NA), _CACHE_ROUND),
    )

def _resolve_offsets(ring_num: int,
                     e_eta_seq: list[float] | None,
                     o_eta_seq: list[float] | None,
                     offsets=None):
    """
    offsets:
      - None: 그대로 사용 (시퀀스가 None이면 0으로 채움)
      - dict: {base_even, base_odd, amp_even, amp_odd, mode, gamma} → make_spiral_offsets 로 생성
      - callable: fn(ring_num) -> (e_seq, o_seq)
    NOTE: make_spiral_offsets(...)는 외부 제공 가정.
    """
    if callable(offsets):
        e_eta_seq, o_eta_seq = offsets(ring_num)
    elif isinstance(offsets, dict):
        # make_spiral_offsets는 외부에서 제공
        e_eta_seq, o_eta_seq = make_spiral_offsets(
            ring_num,
            base_even=offsets.get("base_even", 0.0),
            base_odd =offsets.get("base_odd",  0.0),
            amp_even =offsets.get("amp_even",  0.0),
            amp_odd  =offsets.get("amp_odd",   0.0),
            mode     =offsets.get("mode",     "arch"),
            gamma    =offsets.get("gamma",     1.0),
        )
    # 보정: None이면 0 시퀀스
    if e_eta_seq is None: e_eta_seq = [0.0]*ring_num
    if o_eta_seq is None: o_eta_seq = [0.0]*ring_num
    # 길이 클리핑
    e_eta_seq = list(e_eta_seq)[:ring_num]
    o_eta_seq = list(o_eta_seq)[:ring_num]
    return e_eta_seq, o_eta_seq

def _cache_get(key):
    with _CACHE_LOCK:
        return _OBJ_CACHE.get(key, None)

def _cache_set(key, obj, res):
    with _CACHE_LOCK:
        _OBJ_CACHE[key] = (obj, res)
        # FIFO-ish eviction
        while len(_OBJ_CACHE) > _CACHE_MAX:
            k = next(iter(_OBJ_CACHE))
            _OBJ_CACHE.pop(k, None)

def _cache_drop_by_fname(fname: str):
    """Drop only entries of the given run-tag (f_name)."""
    fname = str(fname)
    with _CACHE_LOCK:
        # f_name is at index 7 in the key tuple
        keys = [k for k in list(_OBJ_CACHE.keys()) if len(k) >= 8 and k[7] == fname]
        for k in keys:
            _OBJ_CACHE.pop(k, None)

def _append_csv(path, row, header=None):
    """
    Append 1 row (thread-safe). Create header if file empty.
    NOTE: open(..., newline='') per Python csv docs to avoid blank lines on Windows.
    """
    try:
        with _FILE_LOCK:
            need_header = (not os.path.exists(path)) or os.path.getsize(path) == 0
            with open(path, 'a', newline='') as f:
                w = csv.writer(f)
                if need_header and header:
                    w.writerow(header)
                w.writerow(row)
    except Exception as e:
        # Never stop optimization due to logging failure
        print(f"[log-warning] csv append failed for {path}: {e}", flush=True)

# ───────────────────────────────────────────────────────────────
# Heavy objective (nm input → convert to m internally)
#   geom(...) must return 'res' dict with keys used below
# ───────────────────────────────────────────────────────────────
def objective_geom(
    x,
    N0: int,
    delta_N: int,
    ring_num: int,
    wavelength_i: float,  # nm
    wavelength_f: float,  # nm
    f_name: str,
    *,
    e_eta_seq: list[float] | None = None,
    o_eta_seq: list[float] | None = None,
    offsets=None,               # dict/callable 지원
    parity_mat: bool = True,
    NA: float = 0.12,
) -> float:
    """Evaluate physical objective for external x=[a_phi[nm], r[nm], rho]. Caches (obj,res).

    Inputs
    -------
    x : array-like length-3 (nm, nm, unitless)
    Returns
    -------
    obj_val : float
      Minimization target. Also stores (obj,res) in the global cache for reuse.
    """
    # 오프셋 확정
    e_eta_seq, o_eta_seq = _resolve_offsets(ring_num, e_eta_seq, o_eta_seq, offsets)

    x = np.asarray(x, float).ravel()
    if x.size != 3:
        raise ValueError("x must be length-3: [a_phi, hole_radius, rho].")
    a_phi, hole_radius, rho_use = x

    key = _obj_key(
        x, N0, delta_N, ring_num, wavelength_i, wavelength_f, f_name,
        e_eta_seq=e_eta_seq, o_eta_seq=o_eta_seq,
        parity_mat=parity_mat, rho=rho_use, NA=NA
    )
    cached = _cache_get(key)
    if cached is not None:
        return cached[0]

    nm = 1e-9
    # geom은 외부 제공. res(dict) 반환 가정.
    res = geom(
        a_phi*nm, hole_radius*nm,
        N0, delta_N, ring_num,
        float(wavelength_i)*nm, float(wavelength_f)*nm,
        e_eta_seq=e_eta_seq, o_eta_seq=o_eta_seq,
        parity_mat=parity_mat, rho=float(rho_use),
        NA=float(NA), f_name=str(f_name)
    )

    CE_f   = float(res['CE_flat'])
    T_norm = float(res['T_norm'])
    pur    = float(res['purcell'])

    # Example composite objective (minimize). Replace if needed.
    obj_val = - np.exp(np.exp(CE_f**(1/np.pi))) * np.log(1 + T_norm) * np.log(pur)
    if not np.isfinite(obj_val):
        raise FloatingPointError("objective became non-finite")

    _cache_set(key, obj_val, res)
    return obj_val

# ───────────────────────────────────────────────────────────────
# Optimization wrapper (external nm; internal y∈[0,1]^3 via Normalizer)
# Replaces SciPy 'minimize' with custom Bridge/Stepper/Gradient stack
# Writes the SAME 4 CSV files (append mode; schema unchanged):
#   - opt_history_{method}_params.csv
#   - out_history_{method}_analysis.csv
#   - opt_history_{method}_obj.csv
#   - zoom_history_{method}.csv
# Returns: OptimizeResult-compatible dict
# ───────────────────────────────────────────────────────────────
def optimize_structure(
    x0,                   # [a_phi, hole_radius, rho]
    N0: int,
    delta_N: int,
    ring_num: int,
    wavelength_i: float,  # nm
    wavelength_f: float,  # nm
    f_name,               # run-tag printed & cached with results
    method='SLSQP',       # kept for preset mapping only (no SciPy path)
    disp=False,           # 기본 OFF
    *,
    e_eta_seq: list[float] | None = None,
    o_eta_seq: list[float] | None = None,
    offsets=None,               # dict/callable 지원
    parity_mat: bool = True,
    rho_bounds: tuple[float, float] = (0.0, 1.0),   # rho 경계(결정변수)
    NA: float = 0.12,
    scale: float | str = "auto"  # [m per optimizer-unit] or "auto" (logging 용도 유지)
):
    """
    Replace SciPy-based inner loop with:
      - Normalizer: x[nm]↔y∈[0,1]^3 (경계 집행)
      - Penalty: hinge^2 for a_phi - k*r >= 0
      - Bridge: 1P bootstrap → 2P 중앙차분
      - Stepper: AdamW×Muon + trust region + improve-only 수용
    I/O/CSV 스키마와 OptimizeResult 호환성은 기존과 동일하게 유지합니다.
    """

    # ── constants & bounds (external nm) ────────────────────────
    nm = 1e-9
    nm_bounds = [(1.2e2, 3.0e2),    # a_phi in [120, 300] nm
                 (4.0e1, 1.5e2)]    # hole_radius in [40, 150] nm
    k = 2.5                         # inequality: a_phi - k*r >= 0

    # ── baseline internal scale (for logging only; same as legacy code) ─
    def _geom_mean(vals):
        vals = np.asarray(vals, float)
        return float(np.exp(np.mean(np.log(np.clip(vals, 1e-300, None)))))
    typ_nm = np.array([np.sqrt(nm_bounds[0][0]*nm_bounds[0][1]),
                       np.sqrt(nm_bounds[1][0]*nm_bounds[1][1])], float)
    auto_scale_base = _geom_mean(typ_nm) * nm
    scale_base = float(auto_scale_base) if (isinstance(scale, str) and str(scale).lower()=="auto") else float(scale)

    # ── CSV paths & headers (unchanged) ─────────────────────────
    params_csv = f'opt_history_{method}_params.csv'
    out_csv    = f'out_history_{method}_analysis.csv'
    obj_csv    = f'opt_history_{method}_obj.csv'
    zoom_csv   = f'zoom_history_{method}.csv'

    params_header = ['f_name','a_phi','hole_radius','N0','delta_N','ring_num',
                     'NA','parity_mat','rho','e_hash','o_hash',
                     'unit_length','scale_m_per_unit','obj']
    out_header = [
        'f_name','a_phi','hole_radius','N0','delta_N','ring_num',
        'lambda','CE_g','CE_l','CE_s','CE_flat','T_norm','purcell',
        'S_0','S_1','S_2','S_3','DoLP','DoCP','best_wl_idx',
        'NA','parity_mat','rho','e_hash','o_hash',
        'unit_length','scale_m_per_unit','obj'
    ]
    obj_header = [
        'f_name','phase','dim','h','a_phi','hole_radius',
        'f_xm','f_x','f_xp','g_fw','g_bw','g_central','g_chosen','scheme',
        'feasible_m','feasible_0','feasible_p','unit_length','scale_m_per_unit'
    ]
    zoom_header = ['f_name','iter','y','s_logL_star','z_logL','L','a_t','a_bar','mode','eff_scale_m_per_unit']

    # ── offsets resolved once per run ───────────────────────────
    e_eta_seq, o_eta_seq = _resolve_offsets(ring_num, e_eta_seq, o_eta_seq, offsets)
    e_hash = _digest_seq(e_eta_seq); o_hash = _digest_seq(o_eta_seq)

    # ── external variable checks ────────────────────────────────
    x0 = np.asarray(x0, float).ravel()
    if x0.size != 3:
        raise ValueError("x0 must be length-3: [a_phi, hole_radius, rho].")
    bounds_ext = [nm_bounds[0], nm_bounds[1], tuple(map(float, rho_bounds))]
    bounds_lo = np.array([b[0] for b in bounds_ext], float)
    bounds_hi = np.array([b[1] for b in bounds_ext], float)

    def _proj_bounds_ext(x_ext):
        x = np.asarray(x_ext, float)
        return np.minimum(np.maximum(x, bounds_lo), bounds_hi)

    def _feasible_ineq(x_ext):
        x = np.asarray(x_ext, float)
        return (x[0] - k*x[1]) >= 0.0

    # ── objective cache hookup (same as legacy) ─────────────────
    def _key_for(x_ext):
        rho_use = float(x_ext[2])
        return _obj_key(
            x_ext, N0, delta_N, ring_num, wavelength_i, wavelength_f, f_name,
            e_eta_seq=e_eta_seq, o_eta_seq=o_eta_seq,
            parity_mat=parity_mat, rho=rho_use, NA=NA
        )

    _last_eval = {'x_ext': None, 'f': None, 'res': None}

    def _ensure_cached(x_ext):
        """Evaluate objective_geom(x_ext, ...) with cache, return (f,res)."""
        x_ext = _proj_bounds_ext(x_ext)
        if _last_eval['x_ext'] is not None and np.allclose(x_ext, _last_eval['x_ext'], rtol=0, atol=0):
            return _last_eval['f'], _last_eval['res']
        entry = _cache_get(_key_for(x_ext))
        if entry is None:
            _ = objective_geom(
                x_ext, N0, delta_N, ring_num, wavelength_i, wavelength_f, f_name,
                e_eta_seq=e_eta_seq, o_eta_seq=o_eta_seq,
                offsets=offsets, parity_mat=parity_mat, NA=NA
            )
            entry = _cache_get(_key_for(x_ext))
        f, res = entry
        _last_eval.update({'x_ext': x_ext.copy(), 'f': f, 'res': res})
        return f, res

    # ── CSV helpers (unchanged) ─────────────────────────────────
    def _log_params_and_analysis(x_ext, obj_used, res, eff_scale):
        unit = "nm"; rho_log = float(x_ext[2])
        _append_csv(params_csv, [
            str(f_name),
            f"{float(x_ext[0]):.6e}", f"{float(x_ext[1]):.6e}",
            int(N0), int(delta_N), int(ring_num),
            f"{float(NA):.6e}", str(bool(parity_mat)), f"{float(rho_log):.6e}",
            e_hash, o_hash, unit, f"{float(eff_scale):.6e}", f"{float(obj_used):.6e}"
        ], header=params_header)

        _append_csv(out_csv, [
            str(f_name),
            f"{float(x_ext[0]):.6e}", f"{float(x_ext[1]):.6e}",
            int(N0), int(delta_N), int(ring_num),
            f"{float(res['lambda']):.6e}",
            f"{float(res['CE_g'] ):.6e}", f"{float(res['CE_l'] ):.6e}",
            f"{float(res['CE_s'] ):.6e}", f"{float(res['CE_flat']):.6e}",
            f"{float(res['T_norm']):.6e}", f"{float(res['purcell']):.6e}",
            f"{float(res['S_0']  ):.6e}", f"{float(res['S_1']  ):.6e}",
            f"{float(res['S_2']  ):.6e}", f"{float(res['S_3']  ):.6e}",
            f"{float(res['DoLP'] ):.6e}", f"{float(res['DoCP'] ):.6e}",
            int(res['best_wl_idx']),
            f"{float(NA):.6e}", str(bool(parity_mat)), f"{float(rho_log):.6e}",
            e_hash, o_hash, unit, f"{float(eff_scale):.6e}", f"{float(obj_used):.6e}"
        ], header=out_header)

    def _log_obj_sample(phase, i, h, x_ext, fm, f0, fp, g_fw, g_bw, g_c, g_chosen, scheme, feas_m, feas_0, feas_p, eff_scale):
        unit = "nm"
        row = [
            str(f_name), str(phase), ("" if i is None else int(i)),
            ("" if h is None else f"{float(h):.6e}"),
            f"{float(x_ext[0]):.6e}", f"{float(x_ext[1]):.6e}",
            ("" if fm is None else f"{float(fm):.6e}"),
            ("" if f0 is None else f"{float(f0):.6e}"),
            ("" if fp is None else f"{float(fp):.6e}"),
            ("" if g_fw is None else f"{float(g_fw):.6e}"),
            ("" if g_bw is None else f"{float(g_bw):.6e}"),
            ("" if g_c  is None else f"{float(g_c ):.6e}"),
            ("" if g_chosen is None else f"{float(g_chosen):.6e}"),
            ("" if scheme is None else str(scheme)),
            str(bool(feas_m)), str(bool(feas_0)), str(bool(feas_p)),
            unit, f"{float(eff_scale):.6e}"
        ]
        _append_csv(obj_csv, row, header=obj_header)

    def _log_zoom(iter_idx, y_vec, mode, eff_scale, a_t=0.0, a_bar=0.0):
        # 내부 zoom은 사용하지 않지만 호환성을 위해 기록
        _append_csv(
            zoom_csv,
            [str(f_name), int(iter_idx),
             f"{float(np.linalg.norm(y_vec)):.6e}",  # y-norm proxy
             f"{0.0:.6e}", f"{0.0:.6e}", f"{1.0:.6e}",  # s_logL*, z_logL, L
             f"{float(a_t):.6e}", f"{float(a_bar):.6e}", str(mode),
             f"{float(eff_scale):.6e}"],
            header=zoom_header
        )

    # ── Normalizer on external bounds (a_phi, r, rho) ───────────
    N = boundary(bounds_ext)            # expects [(L,U), ...]
    # (권장 개선) 시작점 feasible 투영: k*r가 상한을 넘으면 r을 먼저 줄인 뒤 a_phi 보정
    U_a = bounds_ext[0][1]
    if k * x0[1] > U_a:
        x0[1] = min(x0[1], U_a / k)
    # a_phi ≥ k*r 보장
    if x0[0] < k * x0[1]:
        x0 = np.array([np.nextafter(k * x0[1], np.inf), x0[1], x0[2]], float)
    x0 = _proj_bounds_ext(x0)
    y0 = N.encode(x0)                   # y∈[0,1]^3

    # ── penalty (hinge^2), weight adapted at first eval ─────────
    penalty_lambda = None
    def _penalty_from_scale(f_abs):
        # f 스케일에 맞춘 보수적 가중치 (경험적)
        return max(1.0, 10.0 * float(f_abs))

    # ── objective in y-space (called by optimizer) ──────────────
    nfev = 0
    eff_scale = scale_base * 1.0
    def f_y(y):
        """Objective over normalized y. Handles boundary, penalty, logging."""
        nonlocal nfev, penalty_lambda
        y = np.asarray(y, float)
        x_ext = N.decode(y)
        x_ext = _proj_bounds_ext(x_ext)

        f_phys, res = _ensure_cached(x_ext)
        if penalty_lambda is None:
            penalty_lambda = _penalty_from_scale(abs(f_phys))

        g = max(0.0, (k * x_ext[1]) - x_ext[0])  # violation if k*r > a_phi
        f_used = f_phys + penalty_lambda * (g ** 2)

        # CSV logging (per evaluation)
        _log_params_and_analysis(x_ext, f_used, res, eff_scale)
        _last_eval.update({'x_ext': x_ext.copy(), 'f': f_phys, 'res': res})
        nfev += 1
        if disp:
            print(f"{method}: a_phi={x_ext[0]:.6e}, r={x_ext[1]:.6e}, rho={x_ext[2]:.4f}, "
                  f"N0={N0}, dN={delta_N}, ring={ring_num}, obj={f_used:.6e}", flush=True)
        return f_used

    # ── gradient source for Bridge (1P bootstrap → 2P) ─────────
    grad_1p = Forward1P(eps_rel=1e-3, K=2)     # first call only
    grad_2p = Central2P(eps_rel=1e-4, K=6)     # subsequent calls

    # ── configs (presets mapped from 'method') ──────────────────
    method_l = str(method).lower().strip()
    if method_l in ("trust-constr", "trust", "tc"):
        step_cfg = StepperConfig(improve_only=True, initial_delta=0.25, backtrack_max=10)
    elif method_l in ("l-bfgs-b", "lbfgsb", "lbfgs"):
        step_cfg = StepperConfig(improve_only=True, initial_delta=0.15, backtrack_max=12)
    else:  # "slsqp" default-like
        step_cfg = StepperConfig(improve_only=True, initial_delta=0.20, backtrack_max=12)

    bridge_cfg = BridgeConfig(
        seed=None, bootstrap_K1p=2, central_K2p=6,
        eps_rel_1p=1e-3, eps_rel_2p=1e-4
    )
    precond_cfg = PreconditionerConfig(enable=True)

    # ── optimizer construction ──────────────────────────────────
    optimizer = OnePointOptimizer(
        f_fn=f_y, normalizer=N,
        grad_source_boot=grad_1p, grad_source_main=grad_2p,
        bridge_cfg=bridge_cfg, stepper_cfg=step_cfg, precond_cfg=precond_cfg
    )

    # ── run loop (improve-only acceptance; ftol & stall criteria) ─
    maxiter = 300 if method_l == "slsqp" else (200 if method_l.startswith("trust") else 500)
    ftol = 1e-5
    stall_max = 25

    y = y0.copy()
    f_best = f_y(y)            # triggers first eval & sets penalty_lambda
    y_best = y.copy()
    nit = 0
    stall = 0

    _log_zoom(nit, y, mode="NEUTRAL", eff_scale=eff_scale)

    while nit < maxiter:
        nit += 1
        y_new, f_new, info = optimizer.step(y)   # (y_next, f_next, info dict)

        # Improve-only acceptance (external guard)
        if f_new < f_best - ftol * max(1.0, abs(f_best)):
            y, f_best = y_new, f_new
            y_best = y_new.copy()
            stall = 0
            mode = "IN"
        else:
            stall += 1
            mode = "HOLD"

        # zoom-history compatibility log (L=1.0 fixed)
        a_t  = float(info.get("a_t", 0.0)) if isinstance(info, dict) else 0.0
        a_bar= float(info.get("a_bar", 0.0)) if isinstance(info, dict) else 0.0
        _log_zoom(nit, y, mode=mode, eff_scale=eff_scale, a_t=a_t, a_bar=a_bar)

        # stopping rules
        if stall >= stall_max:
            break
        df = info.get("df", None) if isinstance(info, dict) else None
        if (df is not None) and (abs(df) <= ftol * max(1.0, abs(f_best))):
            break

    # ── decode & finalize result ────────────────────────────────
    x_best = N.decode(y_best)
    # 최종 안전 투영(경계/미세 위반 방지)
    x_best = _proj_bounds_ext(x_best)
    if not _feasible_ineq(x_best):
        # 종료 시 미세 투영 (a_phi ← max(a_phi, k*r))
        x_best = np.array([max(x_best[0], k * x_best[1]), x_best[1], x_best[2]], float)

    f_phys_final, _res_final = _ensure_cached(x_best)
    g_final = max(0.0, (k * x_best[1]) - x_best[0])
    penalty_final = (penalty_lambda if penalty_lambda is not None else 1.0) * (g_final ** 2)
    f_used_final = f_phys_final + penalty_final

    # history attach (same as legacy)
    try:
        hist = pd.read_csv(params_csv)
        history_df = hist[hist['f_name'].astype(str) == str(f_name)].reset_index(drop=True)
    except Exception:
        history_df = None

    # SciPy-like result dict
    sol = {
        'x': x_best,
        'fun': float(f_used_final),
        'nit': int(nit),
        'nfev': int(nfev),
        'success': bool(True if stall < stall_max else False),
        'status': int(0 if stall < stall_max else 1),
        'message': "OK: custom optimizer converged" if stall < stall_max else "Stopped: stall limit",
        'history': history_df,
        # extras for downstream debugging
        'bridge': {'method_preset': method_l, 'ftol': ftol, 'stall_max': stall_max},
        'A_info': None
    }

    # cleanup
    _cache_drop_by_fname(f_name)
    return sol


### Sweep operation

In [ ]:
def sweep_params(
    x0=None,                    # [a_phi, hole_radius, rho]  OR list of such seeds
    *,
    # design axes (scalars or lists; lists become sweep axes)
    N0=None,
    delta_N=None,
    ring_num=None,
    wav_lo=None, wav_hi=None,   # in nm
    method='SLSQP',
    # offsets / sequences (scalars or lists; per-run 가능)
    e_eta_seq=None, o_eta_seq=None,   # list[float] or list[list[float]]
    offsets=None,                      # dict/callable or list[…]
    parity_mat: bool | list = True,
    NA: float | list = 0.12,
    scale: float | list | str = "auto",
    # seed options (모두 길이 3이어야 함)
    x0_list=None,               # list of [a_phi, hole_radius, rho]
    x0_grid=None,               # dict {"a_phi":[...nm], "hole_radius":[...nm], "rho":[...]}
    # sweeping control
    grid_mode: str = "product", # {"product","zip"}
    order=None,
    disp: bool = False,
    # parallelization (Ray)
    parallel: str = "none",     # {"none","ray"}
    max_workers: int = 0,
    # selection (opt 모드에서만 사용)
    obj_threshold: float = -50.0,
    tag_template: str | None = None,
    # NEW: 모드 선택
    mode: str = "geom",         # {"geom","opt"}
    # NEW: per-iter opt logs 보존(기본 False; 병렬 경합/손상 방지)
    preserve_opt_logs: bool = False,
):
    """
    Sweep runner (geom/opt).
    - 충돌 방지: sweep 집계 CSV는 'sweep_runs_{method}_{mode}.csv'로 기록.
    - Ray 모드 기본값: optimize_structure의 per-iter CSV는 워커 temp에 남고 폐기.
      preserve_opt_logs=True로 중앙 CSV 4종(opt_history_*.csv, out_history_*.csv, ...)에 병합 가능.
    """
    import os, tempfile, json, csv, shutil, glob
    import numpy as np, pandas as pd
    from itertools import product as _product

    if ring_num is None or wav_lo is None or wav_hi is None:
        raise ValueError("ring_num, wav_lo, wav_hi are required (scalars or lists).")

    mode = str(mode).lower()
    if mode not in {"geom","opt"}:
        raise ValueError("mode must be 'geom' or 'opt'.")

    # ---------- helpers ----------
    def _is_listlike(v):
        return isinstance(v, (list, tuple, np.ndarray))

    def _as_list(v):
        if v is None: return [None]
        return list(v) if _is_listlike(v) else [v]

    def _pad_get(seq, i):
        if not _is_listlike(seq): return seq
        if len(seq) == 0: return None
        return seq[i] if i < len(seq) else seq[-1]

    def _normalize_seeds(x0, x0_list, x0_grid):
        seeds = []
        if x0_grid is not None:
            a_seq = list(x0_grid.get("a_phi", []))
            r_seq = list(x0_grid.get("hole_radius", []))
            rho_seq = list(x0_grid.get("rho", []))
            if not (a_seq and r_seq and rho_seq):
                raise ValueError("x0_grid must provide 'a_phi', 'hole_radius', and 'rho' sequences.")
            seeds = [np.array([a, r, rh], float) for a, r, rh in _product(a_seq, r_seq, rho_seq)]
        if not seeds and x0_list is not None:
            seeds = [np.array(s, float) for s in x0_list]
        if not seeds and x0 is not None:
            if _is_listlike(x0) and len(x0) > 0 and _is_listlike(x0[0]):
                seeds = [np.array(s, float) for s in x0]
            else:
                seeds = [np.array(x0, float)]
        if not seeds:
            raise ValueError("Provide seeds via x0, x0_list, or x0_grid.")
        for s in seeds:
            if s.size != 3:
                raise ValueError("All seeds must be length-3: [a_phi, hole_radius, rho].")
        return seeds

    def _fmt(v):
        try: return f"{float(v):.4g}"
        except: return str(v)

    # external helpers from notebook:
    # _resolve_offsets, _digest_seq, _append_csv, clear_cache, optimize_structure, geom
    if 'geom' not in globals() and mode == 'geom':
        raise RuntimeError("mode='geom' requires a global function geom(...).")

    # ---------- axes digest ----------
    N0_L      = _as_list(N0)
    dN_L      = _as_list(delta_N)
    ring_L    = _as_list(ring_num)
    wav_lo_L  = _as_list(wav_lo)
    wav_hi_L  = _as_list(wav_hi)
    method_L  = _as_list(method)
    NA_L      = _as_list(NA)
    scale_L   = _as_list(scale)
    parity_L  = _as_list(parity_mat)
    e_seq_L   = _as_list(e_eta_seq)
    o_seq_L   = _as_list(o_eta_seq)
    offsets_L = _as_list(offsets)

    seeds = _normalize_seeds(x0, x0_list, x0_grid)

    default_order = ['method','scale','N0','delta_N','ring_num','wav_lo','wav_hi',
                     'NA','parity_mat','e_eta_seq','o_eta_seq','offsets']
    loop_order = order or default_order

    def _axis_values(name):
        return {
            'method':     method_L,
            'scale':      scale_L,
            'N0':         N0_L,
            'delta_N':    dN_L,
            'ring_num':   ring_L,
            'wav_lo':     wav_lo_L,
            'wav_hi':     wav_hi_L,
            'NA':         NA_L,
            'parity_mat': parity_L,
            'e_eta_seq':  e_seq_L,
            'o_eta_seq':  o_seq_L,
            'offsets':    offsets_L,
        }[name]

    tasks = []
    if grid_mode.lower() == "product":
        pools = [(_axis_values(n), n) for n in loop_order]
        for vals in _product(*[p[0] for p in pools]):
            d = {pools[i][1]: vals[i] for i in range(len(pools))}
            tasks.append(d)
    else:  # "zip"
        lengths = [len(_axis_values(n)) for n in default_order if _is_listlike(_axis_values(n))]
        Lmax = max(lengths) if lengths else 1
        for i in range(Lmax):
            d = dict(
                method     = _pad_get(method_L, i),
                scale      = _pad_get(scale_L, i),
                N0         = _pad_get(N0_L, i),
                delta_N    = _pad_get(dN_L, i),
                ring_num   = _pad_get(ring_L, i),
                wav_lo     = _pad_get(wav_lo_L, i),
                wav_hi     = _pad_get(wav_hi_L, i),
                NA         = _pad_get(NA_L, i),
                parity_mat = _pad_get(parity_L, i),
                e_eta_seq  = _pad_get(e_seq_L, i),
                o_eta_seq  = _pad_get(o_seq_L, i),
                offsets    = _pad_get(offsets_L, i),
            )
            tasks.append(d)

    # 태그 템플릿
    if tag_template is None:
        tag_template = "sw_{i}_{method}_N{N0}_dN{dN}_R{R}_a{a}_r{r}_rho{rho}_sc{sc}_NA{NA}_par{par}_eh{eh}_oh{oh}"

    # geom 라벨 보정
    if mode == "geom" and (method_L == ['SLSQP'] or str(method_L[0]).lower() == 'slsqp'):
        method_L = ['geom']

    def _make_tag(i, t, seed, eh, oh):
        return tag_template.format(
            i=i, method=t['method'],
            N0=int(t['N0']), dN=int(t['delta_N']), R=int(t['ring_num']),
            a=_fmt(seed[0]), r=_fmt(seed[1]), rho=_fmt(seed[2]), sc=_fmt(t['scale']),
            NA=_fmt(t['NA']), par=str(bool(t['parity_mat'])),
            eh=str(eh), oh=str(oh),
        )

    # ---- sweep-run aggregated CSV (충돌 방지: 이름 분리) ----
    method0 = str(method_L[0])
    runs_csv = f'sweep_runs_{method0}_{mode}.csv'
    print(f"SWEEP starting: {len(tasks)*len(seeds)} runs (grid_mode={grid_mode}, parallel={parallel}, unit=nm, mode={mode})")

    # Core runner
    def _run_one(seed, t, idx, collect_logs=False, logs_sink_dir=None):
        # offsets/eta 확정
        e_seq_run, o_seq_run = _resolve_offsets(
            int(t['ring_num']),
            t.get('e_eta_seq'), t.get('o_eta_seq'), t.get('offsets')
        )
        eh, oh = _digest_seq(e_seq_run), _digest_seq(o_seq_run)
        ftag = _make_tag(idx, t, seed, eh, oh)

        # per-iter 로그 수집용
        collected = {}

        if mode == "opt":
            sol = optimize_structure(
                np.array(seed, float), int(t['N0']), int(t['delta_N']), int(t['ring_num']),
                float(t['wav_lo']), float(t['wav_hi']),
                f_name=ftag, method=str(t['method']), disp=disp,
                e_eta_seq=e_seq_run, o_eta_seq=o_seq_run, offsets=None,
                parity_mat=bool(t['parity_mat']), NA=float(t['NA']),
                scale=t['scale']
            )
            x = np.array(sol['x'], float) if isinstance(sol, dict) else np.array(sol.x, float)
            obj = float(sol['fun'] if isinstance(sol, dict) else sol.fun)
            res = sol.get('res', {}) if isinstance(sol, dict) else {}
        else:
            nm = 1e-9
            a_phi_nm, hole_nm, rho = map(float, seed)
            res = geom(
                a_phi_nm*nm, hole_nm*nm,
                int(t['N0']), int(t['delta_N']), int(t['ring_num']),
                float(t['wav_lo'])*nm, float(t['wav_hi'])*nm,
                e_eta_seq=e_seq_run, o_eta_seq=o_seq_run,
                parity_mat=bool(t['parity_mat']), rho=float(rho),
                NA=float(t['NA']), f_name=str(ftag)
            )
            x = np.array([a_phi_nm, hole_nm, rho], float)
            obj = np.nan

        # (옵션) 워커 temp → 중앙 병합용 per-iter 로그 수집
        if collect_logs and logs_sink_dir and mode == "opt":
            patterns = [
                f'opt_history_{t["method"]}_params.csv',
                f'out_history_{t["method"]}_analysis.csv',
                f'opt_history_{t["method"]}_obj.csv',
                f'zoom_history_{t["method"]}.csv'
            ]
            for p in patterns:
                if os.path.exists(p):
                    with open(p, 'r', newline='') as f:
                        collected[p] = f.read()

        # 표준 메트릭 추출(있으면 기록)
        CE_flat  = float(res['CE_flat']) if isinstance(res, dict) and 'CE_flat' in res else np.nan
        T_norm   = float(res['T_norm'])  if isinstance(res, dict) and 'T_norm'  in res else np.nan
        purcell  = float(res['purcell']) if isinstance(res, dict) and 'purcell' in res else np.nan

        row = {
            'f_name': ftag,
            'a_phi': float(x[0]), 'hole_radius': float(x[1]),
            'N0': int(t['N0']), 'delta_N': int(t['delta_N']), 'ring_num': int(t['ring_num']),
            'wav_lo': float(t['wav_lo']), 'wav_hi': float(t['wav_hi']),
            'NA': float(t['NA']), 'parity_mat': bool(t['parity_mat']), 'rho': float(x[2]),
            'e_hash': eh, 'o_hash': oh,
            'unit_length': 'nm', 'scale_m_per_unit': np.nan,
            'obj': obj,
            'CE_flat': CE_flat, 'T_norm': T_norm, 'purcell': purcell,
            'res_json': json.dumps(res, default=float) if isinstance(res, dict) else "",
        }
        return row, collected

    # ---- 실행 (단일 or Ray) ----
    results = []
    collected_logs = []  # (파일명, 내용)
    header = ['f_name','a_phi','hole_radius','N0','delta_N','ring_num',
              'wav_lo','wav_hi','NA','parity_mat','rho','e_hash','o_hash',
              'unit_length','scale_m_per_unit','obj','CE_flat','T_norm','purcell','res_json']

    if parallel.lower() == "ray":
        try:
            import ray
        except Exception as e:
            raise RuntimeError("parallel='ray' requested but Ray is not installed.") from e
        if not ray.is_initialized():
            ray.init(ignore_reinit_error=True, include_dashboard=False,
                     num_cpus=(None if max_workers <= 0 else max_workers))

        @ray.remote
        def _ray_worker(seed, t, idx, preserve_logs):
            import numpy as _np, os, tempfile, json, csv
            # temp 디렉토리에서 충돌 없이 실행
            with tempfile.TemporaryDirectory() as _td:
                _cwd = os.getcwd()
                try:
                    os.chdir(_td)
                    row, logs = _run_one(_np.array(seed, float), t, idx,
                                         collect_logs=bool(preserve_logs),
                                         logs_sink_dir=_cwd)
                    return row, logs
                finally:
                    os.chdir(_cwd)

        futs, idx = [], 0
        for t in tasks:
            if t.get('method') is None:
                t['method'] = method_L[0]
            for s in seeds:
                futs.append(_ray_worker.remote(np.array(s, float), dict(t), idx, preserve_opt_logs))
                idx += 1
        for fut in futs:
            row, logs = ray.get(fut)
            results.append(row)
            if preserve_opt_logs and logs:
                for k, v in logs.items():
                    collected_logs.append((k, v))
    else:
        idx = 0
        for t in tasks:
            if t.get('method') is None:
                t['method'] = method_L[0]
            for s in seeds:
                row, logs = _run_one(np.array(s, float), t, idx,
                                     collect_logs=False, logs_sink_dir=None)
                results.append(row)
                idx += 1

    # ---- sweep 집계 CSV에 기록(충돌 방지된 단일 파일) ----
    if results:
        need_header = (not os.path.exists(runs_csv)) or os.path.getsize(runs_csv) == 0
        with open(runs_csv, 'a', newline='') as f:
            w = csv.writer(f)
            if need_header:
                w.writerow(header)
            for row in results:
                w.writerow([row.get(k, "") for k in header])

    df = pd.DataFrame(results, columns=header)

    # 후보(candidates) 선별
    if mode == "opt":
        df['obj'] = pd.to_numeric(df['obj'], errors='coerce')
        df = df.dropna(subset=['obj'])
        best_idx = df.groupby('f_name')['obj'].idxmin()
        reduced = df.loc[best_idx].reset_index(drop=True)
        candidates = reduced[reduced['obj'] <= obj_threshold].copy()
    else:
        candidates = df.reset_index(drop=True)

    out_csv = f'sweep_candidates_{method0}_{mode}.csv'
    candidates.to_csv(out_csv, index=False)
    print(f"\nSaved sweep runs to {runs_csv} (rows={len(df)})", flush=True)
    print(f"Saved candidates to {out_csv} (rows={len(candidates)})", flush=True)

    # (옵션) per-iter 로그 중앙 병합
    if preserve_opt_logs and collected_logs:
        # 파일명별로 헤더 중복 제거하며 append
        def _append_raw_lines(dst, raw_text):
            os.makedirs(os.path.dirname(dst), exist_ok=True) if os.path.dirname(dst) else None
            write_header = (not os.path.exists(dst)) or os.path.getsize(dst) == 0
            lines = raw_text.splitlines()
            if not lines:
                return
            with open(dst, 'a', newline='') as f:
                if write_header:
                    f.write(lines[0] + '\n')
                for ln in (lines[1:] if write_header else lines[1:]):
                    if ln.strip():
                        f.write(ln + '\n')

        # 대상 파일 4종
        for fname, raw in collected_logs:
            _append_raw_lines(fname, raw)
        print("Merged per-iter optimize logs from workers into central CSVs.", flush=True)

    clear_cache()
    return candidates


- 단일 최적화
sol = optimize_structure(
    x0=[180.0, 60.0, 0.5],     # 길이 3 고정
    N0=18, delta_N=14, ring_num=14,
    wavelength_i=1270.0, wavelength_f=1320.0,
    f_name="demo_len3",
    offsets=dict(amp_even=0.12, mode="arch"),
    disp=False
)


- 스윕 (Ray 병렬 예)
winners = sweep_params(
    x0_grid={"a_phi":[160,180,200], "hole_radius":[55,65], "rho":[0.3,0.5,0.7]},
    N0=[16,18], delta_N=[10,12], ring_num=14,
    wav_lo=1270.0, wav_hi=1320.0,
    offsets=[dict(amp_even=0.10,mode="arch"), dict(amp_even=0.15,mode="arch")],
    parallel="ray", max_workers=8, disp=False
)


In [ ]:
candidates = sweep_params(
    x0_grid={"a_phi":[211.1827], "hole_radius":[75.23277],
             "rho": [0.492]},
    N0=[18], delta_N=[14], ring_num=[14],
    wav_lo=[1270.0], wav_hi=[1320.0],
    e_eta_seq=[0],
    offsets=None, parity_mat=[True], NA=[0.12], scale=["auto"],
    method="trust-constr",   
    mode="opt",               
    obj_threshold=1e9,        
    grid_mode="product",
    parallel="none", disp=False
)
print(candidates[['f_name','obj','CE_flat','T_norm','purcell']].head())

In [ ]:
candidates = sweep_params(
    x0_grid={"a_phi":[211.1827], "hole_radius":[75.23277],
             "rho": np.arange(0.490, 0.494, 0.0005)},
    N0=[18], delta_N=[14], ring_num=[14],
    wav_lo=[1270.0], wav_hi=[1320.0],
    e_eta_seq=[1/3, 2/3],      
    offsets=None, parity_mat=[True], NA=[0.12], scale=["auto"],
    method="geom",             
    mode="geom",
    grid_mode="product", parallel="none", disp=False
)
print(candidates[['f_name','CE_flat','T_norm','purcell']].head())


SWEEP starting: 9 runs (grid_mode=product, parallel=none, unit=nm, mode=geom)
